In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [3]:
init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    init.run()
    result = f.eval()

In [5]:
result

42

In [6]:
sess = tf.InteractiveSession()

In [7]:
init.run()

In [8]:
result = f.eval()

In [9]:
result

42

In [10]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [11]:
housing = fetch_california_housing()
m,n = housing.data.shape
housing_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [12]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [13]:
X = tf.constant(housing_plus_bias, dtype=tf.float32, name="X")

In [14]:
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")

In [15]:
XT = tf.transpose(X)

In [16]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [17]:
with tf.Session() as sess:
    theta_value = theta.eval() 

In [20]:
theta_value

array([[-3.7226696e+01],
       [ 4.3558651e-01],
       [ 9.3845110e-03],
       [-1.0583513e-01],
       [ 6.3847488e-01],
       [-4.1102558e-06],
       [-3.7780956e-03],
       [-4.2437303e-01],
       [-4.3785861e-01]], dtype=float32)

In [85]:
from sklearn.preprocessing import StandardScaler

In [86]:
n_epochs = 1000
learning_rate = 0.1

sc = StandardScaler()

In [87]:
scaled_housing_plus_bias = sc.fit_transform(housing_plus_bias)

In [88]:
X = tf.constant(scaled_housing_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [89]:
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")

In [90]:
y_pred = tf.matmul(X, theta, name="predictions")

In [91]:
error = y_pred - y

In [92]:
mse = tf.reduce_mean(tf.square(error), name="mse")

In [93]:
gradients = tf.gradients(mse, [theta])[0]

In [103]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [104]:
training_op = optimizer.minimize(mse)

In [105]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("epoch: ", epoch, " MSE: ", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()

epoch:  0  MSE:  7.4134116
epoch:  100  MSE:  4.8051915
epoch:  200  MSE:  4.803446
epoch:  300  MSE:  4.8032804
epoch:  400  MSE:  4.803258
epoch:  500  MSE:  4.8032546
epoch:  600  MSE:  4.803254
epoch:  700  MSE:  4.8032537
epoch:  800  MSE:  4.803254
epoch:  900  MSE:  4.803254


In [107]:
best_theta

array([[ 0.9992008 ],
       [ 0.8296098 ],
       [ 0.11874986],
       [-0.26550898],
       [ 0.3056814 ],
       [-0.00450353],
       [-0.03932592],
       [-0.8999076 ],
       [-0.87056184]], dtype=float32)

In [113]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [114]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [115]:
my_func(0.2, 0.3)

-0.21253923284754914

In [116]:
reset_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")
for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)

grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [117]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671394]
